In [1]:
import os

In [2]:
%pwd

'd:\\Machine Learning project\\classifier\\classifier\\research'

In [3]:
os.chdir("..")

In [4]:
%pwd

'd:\\Machine Learning project\\classifier\\classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.classifier.constant import *
from src.classifier.logger import logging
from src.classifier.exception import ModelException
from src.classifier.utils.common import read_yaml,create_directory

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(path_to_yaml=Path("config/config.yaml"))
        #self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingenstion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os,sys
import urllib.request as request
import gdown
import zipfile
from src.classifier.exception import ModelException
from src.classifier.logger import logging



In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logging.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logging.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
a = read_yaml(path_to_yaml=Path("config/config.yaml"))

In [15]:
a

ConfigBox({'artifacts_root': 'artifacts', 'data_ingenstion': {'root_dir': 'artifacts/data_ingenstion', 'source_url': 'https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing', 'local_data_file': 'artifacts/data_ingenstion/data.zip', 'unzip_dir': 'artifacts/data_ingenstion'}})

In [16]:
#a = read_yaml(CONFIG_FILE_PATH)

In [17]:
a.data_ingenstion

ConfigBox({'root_dir': 'artifacts/data_ingenstion', 'source_url': 'https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing', 'local_data_file': 'artifacts/data_ingenstion/data.zip', 'unzip_dir': 'artifacts/data_ingenstion'})

In [18]:
create_directory([a.data_ingenstion.root_dir])

In [19]:
%pwd

'd:\\Machine Learning project\\classifier\\classifier'

In [22]:
try:
    config = ConfigurationManager()
    data_ingenstion_config = config.get_data_ingestion_config()
    data_ingenstion = DataIngestion(config=data_ingenstion_config)
    data_ingenstion.download_file()
    data_ingenstion.extract_zip_file()
except Exception as e:
    raise ModelException(e,sys)

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=cf54953c-fbed-4695-b811-c460e808da56
To: d:\Machine Learning project\classifier\classifier\artifacts\data_ingenstion\data.zip
100%|██████████| 57.7M/57.7M [01:34<00:00, 613kB/s] 


In [14]:
%pwd

'd:\\Machine Learning project\\classifier\\classifier'

In [ ]:
%pwd

'd:\\Machine Learning project\\classifier'